# Make MTH5 from legacy Phoenix MTU 

Older MTU units output data in a different format than the newer MTU-5C units.  The data are stored in `.TSN` files where the `N` represents the sample rate, these files are accompanied by a `.TBL` file which contains the metadata needed to characterize the data.  Inside each `.TSN` file is the data for all channels recorded at the sample rate.    

Thanks to Dong Hao for his Python code, a reader for the `.TBL` and `.TSN` files is now included in `mth5`. The reader output is a `RunTS` object.

<div class="alert alert-block alert-warning">
<b>TODO:</b> update `PhoenixCollection` to accommodate the older file format and then update `PhoenixCollection.assign_run_names` to reflect the older format and expected heirarchy. Then `MakeMTH5.from_phoenix` can be used. 
</div>  

<div class="alert alert-block alert-info">
<b>Note:</b> This example assumes that mth5_test_data is installed
</div>

## Imports

In [1]:

from pathlib import Path
from mth5_test_data import get_test_data_path
from mth5.io.phoenix import read_phoenix
from mth5.mth5 import MTH5


## Example Data

These data come courtesy of Dong Hao.  They reside in `mth5_test_data.phoenix_mtu`.  Lets load in the path and have a look at the files.

You will see there are 3 `TSN` files one for each sample rate, and one `TBL` file that provides the metadata information about the station recording.

In [2]:
mtu_path = get_test_data_path("phoenix_mtu")

In [3]:
ts_files = []
for fn in mtu_path.iterdir():
    print(fn.name)
    if fn.suffix.upper() in [".TS2", ".TS3", ".TS4", ".TS5", ".TSL", ".TSH"]:
        ts_files.append(fn)
    elif fn.suffix.upper() == ".TBL":
        mtu_table_filepath = fn

1690C16C.TBL
1690C16C.TS3
1690C16C.TS4
1690C16C.TS5
phoenix_mtu_test_data.zip


### Table File
The `TBL` file provides the metadata for the station recording.  This information has been crudely translated to `mt-metadata.timeseries` objects.  The metadata key names are up to 4 characters and are therefore difficult to translate without the proper documentation.  Therefore, if you have more knowledge about the metadata in the `TBL` file could you either contribute or raise an issue and provide more details.

In [4]:
mtu_table = read_phoenix(mtu_table_filepath)

2026-01-03T14:42:33.515293-0800 | WARNING | mth5.io.phoenix.readers.mtu.mtu_table | _get_dictionary_from_tbl | line: 330 | Failed to decode HW: unpack requires a buffer of 16 bytes, storing raw bytes


#### Survey Metadata
Have a look at the `mt_metadata.timeseries.Survey` object.  As you can see the metadata is minimal.  The user should input the `id` at the very least.

In [5]:
print(mtu_table.survey_metadata.to_json(required=True))

ValidationError: 1 validation error for Software
version
  Input should be a valid string, unable to parse raw data as a unicode string [type=string_unicode, input_value=b'MTU52\x003100\xecQ\xd2', input_type=bytes]
    For further information visit https://errors.pydantic.dev/2.10/v/string_unicode

#### Station Metadata

Have a look at the `mt_metadata.timeseries.Station` object.  Notice there is a default run in the `run_list` this should be adjusted when creating an `MTH5`.

In [ ]:
print(mtu_table.station_metadata.to_json(required=True))

{
    "station": {
        "acquired_by.author": "",
        "channel_layout": "X",
        "channels_recorded": [
            "ex",
            "ey",
            "hx",
            "hy",
            "hz"
        ],
        "data_type": "BBMT",
        "geographic_name": "",
        "id": "10441W10",
        "location.datum": "WGS 84",
        "location.declination.model": "IGRF",
        "location.declination.value": 0.0,
        "location.elevation": 1304.0,
        "location.elevation_uncertainty": 0.0,
        "location.latitude": 41.00388,
        "location.latitude_uncertainty": 0.0,
        "location.longitude": 104.00536,
        "location.longitude_uncertainty": 0.0,
        "location.x": 0.0,
        "location.x2": 0.0,
        "location.x_uncertainty": 0.0,
        "location.y": 0.0,
        "location.y2": 0.0,
        "location.y_uncertainty": 0.0,
        "location.z": 0.0,
        "location.z2": 0.0,
        "location.z_uncertainty": 0.0,
        "orientation.method": "com

#### Run Metadata

Have a look at the `mt_metadata.timeseries.Run` object.  

In [ ]:
print(mtu_table.run_metadata.to_json(required=True))

{
    "run": {
        "acquired_by.author": "",
        "channels_recorded_auxiliary": [],
        "channels_recorded_electric": [
            "ex",
            "ey"
        ],
        "channels_recorded_magnetic": [
            "hx",
            "hy",
            "hz"
        ],
        "data_logger.firmware.author": "",
        "data_logger.firmware.name": "",
        "data_logger.firmware.version": "MTU52\u00003100\u00ecQ\u00d2",
        "data_logger.id": "MTU_1690",
        "data_logger.power_source.voltage.end": 0.0,
        "data_logger.power_source.voltage.start": 0.0,
        "data_logger.timing_system.drift": 0.0,
        "data_logger.timing_system.n_satellites": 8,
        "data_logger.timing_system.type": "GPS",
        "data_logger.timing_system.uncertainty": 0.0,
        "data_type": "BBMT",
        "id": "run_1690",
        "metadata_by.author": "",
        "provenance.archive.name": "",
        "provenance.creation_time": "1980-01-01T00:00:00+00:00",
        "provenance

#### Channel Metadata

Each channel has its own channel object.  Note that these are generic and the sample rate will be updated when the time series is read from a `TSN` file. As will the start time and end time.

In [ ]:
for ch in ["ex", "ey", "hx", "hy", "hz"]:
    print(f"{'='*20} {ch} {'='*20}")
    print(getattr(mtu_table, f"{ch}_metadata").to_json(required=True))

==================== ex ====================
{
    "electric": {
        "ac.end": 0.0,
        "ac.start": 0.0005017281176719806,
        "channel_number": 1,
        "component": "ex",
        "contact_resistance.end": 0.0,
        "contact_resistance.start": 0.0,
        "data_quality.rating.value": null,
        "dc.end": 0.0,
        "dc.start": -0.009348183792613004,
        "dipole_length": 100.0,
        "filters": [],
        "measurement_azimuth": 0.0,
        "measurement_tilt": 0.0,
        "negative.datum": "WGS 84",
        "negative.elevation": 0.0,
        "negative.elevation_uncertainty": 0.0,
        "negative.latitude": 0.0,
        "negative.latitude_uncertainty": 0.0,
        "negative.longitude": 0.0,
        "negative.longitude_uncertainty": 0.0,
        "negative.x": 0.0,
        "negative.x2": 0.0,
        "negative.x_uncertainty": 0.0,
        "negative.y": 0.0,
        "negative.y2": 0.0,
        "negative.y_uncertainty": 0.0,
        "negative.z": 0.0,
     

## Create MTH5

Now we will demonstrate how to create an MTH5 from the legacy MTU Phoenix data.

In [ ]:
with MTH5() as m:
    m = m.open_mth5(Path.cwd() / "example_mtu.mth5", mode="a")

    # step 1: add a survey group
    survey_metadata = mtu_table.survey_metadata
    survey_metadata.id = "example_mtu_survey"
    survey_group = m.add_survey(survey_metadata.id, survey_metadata=survey_metadata)

    # step 2: add a station group
    station_metadata = mtu_table.station_metadata
    station_metadata.remove_run("run_1690")
    station_group = survey_group.stations_group.add_station(
        station_metadata.id, station_metadata=station_metadata
    )

    # step 3: add runs
    for fn in ts_files:
        print(f"Adding {fn.name}")
        sr_extension = fn.suffix[-1]
        run_ts = read_phoenix(fn, **{"table_filepath": mtu_table_filepath})
        print(f"Run name: {run_ts.run_metadata.id}, SR: {run_ts.run_metadata.sample_rate} Hz")
        print(run_ts)
        # add Run
        run_group = station_group.add_run(run_ts.run_metadata.id, run_metadata=run_ts.run_metadata)
        run_group.from_runts(run_ts)

    m.channel_summary.summarize()
    channel_df = m.channel_summary.to_dataframe()
    run_df = m.run_summary()
    print(m)

2026-01-03T14:35:30.237150-0800 | INFO | mth5.groups.survey | add_survey | line: 281 | survey example_mtu_survey already exists, returning existing group.
2026-01-03T14:35:30.295178-0800 | INFO | mth5.groups.base | _add_group | line: 330 | StationGroup 10441W10 already exists, returning existing group.
Adding 1690C16C.TS3
2026-01-03T14:35:30.345518-0800 | INFO | mth5.io.phoenix.readers.mtu.mtu_ts | read | line: 513 | Opening file: 1690C16C.TS3 ...
2026-01-03T14:35:30.346513-0800 | INFO | mth5.io.phoenix.readers.mtu.mtu_ts | _read_header | line: 267 | Start time is: 2009-12-16T07:55:00+00:00 UTC
2026-01-03T14:35:30.346513-0800 | INFO | mth5.io.phoenix.readers.mtu.mtu_ts | _read_header | line: 289 | Sampling frequency is 2400 Hz
2026-01-03T14:35:30.347508-0800 | INFO | mth5.io.phoenix.readers.mtu.mtu_ts | _read_header | line: 290 | Number of records is 2400 in each data block
2026-01-03T14:35:30.347508-0800 | INFO | mth5.io.phoenix.readers.mtu.mtu_ts | _read_header | line: 292 | TS type 

ValueError: VLEN strings do not support embedded NULLs